In [37]:
# Data source:
# https://www.kaggle.com/datasets/heesoo37/120-years-of-olympic-history-athletes-and-results

# Import packages

In [38]:
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm

# Load source data

In [39]:
df = pd.read_csv('OlympicHistory/athlete_events.csv')

In [40]:
df.head()

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal
0,1,A Dijiang,M,24.0,180.0,80.0,China,CHN,1992 Summer,1992,Summer,Barcelona,Basketball,Basketball Men's Basketball,NaN
1,2,A Lamusi,M,23.0,170.0,60.0,China,CHN,2012 Summer,2012,Summer,London,Judo,Judo Men's Extra-Lightweight,NaN
2,3,Gunnar Nielsen Aaby,M,24.0,NaN,NaN,Denmark,DEN,1920 Summer,1920,Summer,Antwerpen,Football,Football Men's Football,NaN
3,4,Edgar Lindenau Aabye,M,34.0,NaN,NaN,Denmark/Sweden,DEN,1900 Summer,1900,Summer,Paris,Tug-Of-War,Tug-Of-War Men's Tug-Of-War,Gold
4,5,Christine Jacoba Aaftink,F,21.0,185.0,82.0,Netherlands,NED,1988 Winter,1988,Winter,Calgary,Speed Skating,Speed Skating Women's 500 metres,NaN


# Data initial analysis

In [41]:
df.shape

(271116, 15)

In [42]:
df.info()
# There are null values in [Age, Height, Weight, Medal]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 271116 entries, 0 to 271115
Data columns (total 15 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   ID      271116 non-null  int64  
 1   Name    271116 non-null  object 
 2   Sex     271116 non-null  object 
 3   Age     261642 non-null  float64
 4   Height  210945 non-null  float64
 5   Weight  208241 non-null  float64
 6   Team    271116 non-null  object 
 7   NOC     271116 non-null  object 
 8   Games   271116 non-null  object 
 9   Year    271116 non-null  int64  
 10  Season  271116 non-null  object 
 11  City    271116 non-null  object 
 12  Sport   271116 non-null  object 
 13  Event   271116 non-null  object 
 14  Medal   39783 non-null   object 
dtypes: float64(3), int64(2), object(10)
memory usage: 31.0+ MB


# Drop inappropriate columns

In [43]:
# Drop columns ['Games']

In [44]:
df = df.drop(columns = ['Games'], inplace = False)

In [45]:
df.shape

(271116, 14)

In [46]:
df.head()

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Year,Season,City,Sport,Event,Medal
0,1,A Dijiang,M,24.0,180.0,80.0,China,CHN,1992,Summer,Barcelona,Basketball,Basketball Men's Basketball,NaN
1,2,A Lamusi,M,23.0,170.0,60.0,China,CHN,2012,Summer,London,Judo,Judo Men's Extra-Lightweight,NaN
2,3,Gunnar Nielsen Aaby,M,24.0,NaN,NaN,Denmark,DEN,1920,Summer,Antwerpen,Football,Football Men's Football,NaN
3,4,Edgar Lindenau Aabye,M,34.0,NaN,NaN,Denmark/Sweden,DEN,1900,Summer,Paris,Tug-Of-War,Tug-Of-War Men's Tug-Of-War,Gold
4,5,Christine Jacoba Aaftink,F,21.0,185.0,82.0,Netherlands,NED,1988,Winter,Calgary,Speed Skating,Speed Skating Women's 500 metres,NaN


# Check structural errors

In [47]:
for i in df.columns:
    print(f"{'Types of '}{i:<8}{': '}{len(df[i].unique())}")

Types of ID      : 135571
Types of Name    : 134732
Types of Sex     : 2
Types of Age     : 75
Types of Height  : 96
Types of Weight  : 221
Types of Team    : 1184
Types of NOC     : 230
Types of Year    : 35
Types of Season  : 2
Types of City    : 42
Types of Sport   : 66
Types of Event   : 765
Types of Medal   : 4


In [48]:
# ID > Name, means that there are some people has the same name (full name)
diff = len(df.ID.unique()) - len(df.Name.unique())
diff
# There are 839 individuals has the same full name

839

# Fill missing data

In [49]:
# As mentioned before, there are null values in [Age, Height, Weight, Medal]
# Null percentage: 4.7% for Age, 25.0% for Height, 25.7% for Weight, 85.9% for Medal.

## Age

In [50]:
# Missing age will be filled by median age of athletes from the corresponding type of sport.
# Take gender in consideration.
medianAgeInSport = df[['Age', 'Sex','Sport']].groupby(by = ['Sport', 'Sex'], as_index = False).median().sort_values('Age')
medianAgeInSport

,Sport,Sex,Age
50,Gymnastics,F,18.0
72,Rhythmic Gymnastics,F,18.0
94,Swimming,F,19.0
87,Ski Jumping,F,19.0
42,Figure Skating,F,20.0
...,...,...,...
28,Croquet,F,39.5
7,Art Competitions,F,42.0
3,Alpinism,F,43.0
8,Art Competitions,M,45.0


In [51]:
for i in tqdm(range(len(df))):
    if pd.isnull(df.Age[i]):
        df.loc[i, 'Age'] = int(medianAgeInSport[(medianAgeInSport.Sport == df.Sport[i]) & (medianAgeInSport.Sex == df.Sex[i])].Age.values)

  0%|          | 0/271116 [00:00<?, ?it/s]

## Height

In [52]:
# Missing height will be filled by median height of athletes from the corresponding type of sport.
# Take gender in consideration.

medianHeightInSport = df[['Height', 'Sex', 'Sport']].groupby(by = 'Sport', as_index = False).median().sort_values('Height')
medianHeightInSport

,Sport,Height
27,Gymnastics,164.0
20,Diving,167.0
59,Trampolining,167.0
23,Figure Skating,168.0
64,Weightlifting,168.0
...,...,...
9,Basque Pelota,NaN
15,Cricket,NaN
16,Croquet,NaN
35,Military Ski Patrol,NaN


In [53]:
len(medianHeightInSport[pd.isnull(medianHeightInSport.Height)])

7

In [54]:
# Considering there are 7 types of sport did not record Height information
# we set them to the average value of all atheletes with the same gender
medianHeightInSport[pd.isnull(medianHeightInSport.Height)]

,Sport,Height
0,Aeronautics,NaN
2,Alpinism,NaN
9,Basque Pelota,NaN
15,Cricket,NaN
16,Croquet,NaN
35,Military Ski Patrol,NaN
42,Roque,NaN


In [55]:
heightOfMale = int(df[df.Sex == 'M'].Height.mean())
heightOfFemale = int(df[df.Sex == 'F'].Height.mean())

for i in tqdm(range(len(df))):
    if pd.isnull(df.Height[i]):
        currentSport = df.Sport[i]
        # if this sport did not record height
        if (pd.isnull(medianHeightInSport.Height[medianHeightInSport.Sport == currentSport])).bool():
            if df.loc[i, 'Sex'] == 'M':
                df.loc[i, 'Height'] = heightOfMale
            else:
                df.loc[i, 'Height'] = heightOfFemale
        # if this sport recorded height
        else:
            df.loc[i, 'Height'] = int(medianHeightInSport[medianHeightInSport.Sport == df.Sport[i]].Height.values)

  0%|          | 0/271116 [00:00<?, ?it/s]

## Weight

In [56]:
# Missing weight will be filled by median weight of athletes from the corresponding type of sport.
# Take gender in consideration.

medianweightInSport = df[['Weight', 'Sex', 'Sport']].groupby(by = 'Sport', as_index = False).median().sort_values('Weight')

In [57]:
weightOfMale = int(df[df.Sex == 'M'].Weight.mean())
weightOfFemale = int(df[df.Sex == 'F'].Weight.mean())

for i in tqdm(range(len(df))):
    if pd.isnull(df.Weight[i]):
        currentSport = df.Sport[i]
        # if this sport did not record weight
        if (pd.isnull(medianweightInSport.Weight[medianweightInSport.Sport == currentSport])).bool():
            if df.loc[i, 'Sex'] == 'M':
                df.loc[i, 'Weight'] = weightOfMale
            else:
                df.loc[i, 'Weight'] = weightOfFemale
        # if this sport recorded weight
        else:
            df.loc[i, 'Weight'] = int(medianweightInSport[medianweightInSport.Sport == df.Sport[i]].Weight.values)

  0%|          | 0/271116 [00:00<?, ?it/s]

## Medal

In [58]:
# Considering over 85% of paticipants did not win a medal
# we set all of them to 'Thanks' instead of null

In [59]:
df.Medal = df.Medal.fillna('Thanks')

# Count & Drop duplicates

## Count sport number for each individual

In [60]:
# Insteand of drop duplicates, we added two columns ['AmountOfSport', 'AmountOfEvent']

In [61]:
countSportsEachId = df[['ID', 'Sport']].groupby(by = 'ID', as_index = False).nunique()
countSportsEachId = countSportsEachId.set_index('ID')
countSportsEachId = countSportsEachId.rename(columns = {"Sport": "AOS"})
df = df.join(countSportsEachId, on = 'ID')
df.head()

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Year,Season,City,Sport,Event,Medal,AOS
0,1,A Dijiang,M,24.0,180.0,80.0,China,CHN,1992,Summer,Barcelona,Basketball,Basketball Men's Basketball,Thanks,1
1,2,A Lamusi,M,23.0,170.0,60.0,China,CHN,2012,Summer,London,Judo,Judo Men's Extra-Lightweight,Thanks,1
2,3,Gunnar Nielsen Aaby,M,24.0,175.0,71.0,Denmark,DEN,1920,Summer,Antwerpen,Football,Football Men's Football,Thanks,1
3,4,Edgar Lindenau Aabye,M,34.0,182.0,95.0,Denmark/Sweden,DEN,1900,Summer,Paris,Tug-Of-War,Tug-Of-War Men's Tug-Of-War,Gold,1
4,5,Christine Jacoba Aaftink,F,21.0,185.0,82.0,Netherlands,NED,1988,Winter,Calgary,Speed Skating,Speed Skating Women's 500 metres,Thanks,1


In [62]:
# countSportsEachId.Sport.unique()
# # some individuals took part in over 1 sport (2-4 sports)

## Count event number for each individual

In [63]:
countEventsEachId = df[['ID', 'Event']].groupby(by = 'ID', as_index = False).nunique()
countEventsEachId = countEventsEachId.set_index('ID')
countEventsEachId = countEventsEachId.rename(columns = {"Event": "AOE"})
df = df.join(countEventsEachId, on = 'ID')
df.head()

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Year,Season,City,Sport,Event,Medal,AOS,AOE
0,1,A Dijiang,M,24.0,180.0,80.0,China,CHN,1992,Summer,Barcelona,Basketball,Basketball Men's Basketball,Thanks,1,1
1,2,A Lamusi,M,23.0,170.0,60.0,China,CHN,2012,Summer,London,Judo,Judo Men's Extra-Lightweight,Thanks,1,1
2,3,Gunnar Nielsen Aaby,M,24.0,175.0,71.0,Denmark,DEN,1920,Summer,Antwerpen,Football,Football Men's Football,Thanks,1,1
3,4,Edgar Lindenau Aabye,M,34.0,182.0,95.0,Denmark/Sweden,DEN,1900,Summer,Paris,Tug-Of-War,Tug-Of-War Men's Tug-Of-War,Gold,1,1
4,5,Christine Jacoba Aaftink,F,21.0,185.0,82.0,Netherlands,NED,1988,Winter,Calgary,Speed Skating,Speed Skating Women's 500 metres,Thanks,1,2


## Add a new column "YearOfBirth"

In [66]:
df['YOB'] = df['Year'] - df['Age']
df.YOB = df.YOB.astype('int64')

## Drop duplicates

In [67]:
# Drop the duplicates with the same ID, Age, Year, Sport
# that means we keep the duplicate individuals in the different sports
df = df.drop_duplicates(subset = ['ID', 'Age', 'Year', 'Sport'])

# Validation

In [68]:
# This is a good example for initial missing data validation, this guy lacks of age, height, weight and medal. LOL.
# After we filled the missing data, all of them are filled as we expected.
# df[df.ID == 58]

In [69]:
# Complete null check
def nullCheck():
    print("----------Start Null Check----------")
    for column in df.columns:
        if len(df[df[column].isnull()]) == 0:
            print(f"{column:<10}{'does not have null value'}")
        else:
            nullId = df.ID[df[column].isnull()]
            print("Null value ID is ", [df[column].isnull()])
    print("----------End Null Check----------")

In [70]:
nullCheck()

----------Start Null Check----------
ID        does not have null value
Name      does not have null value
Sex       does not have null value
Age       does not have null value
Height    does not have null value
Weight    does not have null value
Team      does not have null value
NOC       does not have null value
Year      does not have null value
Season    does not have null value
City      does not have null value
Sport     does not have null value
Event     does not have null value
Medal     does not have null value
AOS       does not have null value
AOE       does not have null value
YOB       does not have null value
----------End Null Check----------


In [71]:
df.to_csv('OlympicHistory/CleanedData.csv', index = False)
df

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Year,Season,City,Sport,Event,Medal,AOS,AOE,YOB
0,1,A Dijiang,M,24.0,180.0,80.0,China,CHN,1992,Summer,Barcelona,Basketball,Basketball Men's Basketball,Thanks,1,1,1968
1,2,A Lamusi,M,23.0,170.0,60.0,China,CHN,2012,Summer,London,Judo,Judo Men's Extra-Lightweight,Thanks,1,1,1989
2,3,Gunnar Nielsen Aaby,M,24.0,175.0,71.0,Denmark,DEN,1920,Summer,Antwerpen,Football,Football Men's Football,Thanks,1,1,1896
3,4,Edgar Lindenau Aabye,M,34.0,182.0,95.0,Denmark/Sweden,DEN,1900,Summer,Paris,Tug-Of-War,Tug-Of-War Men's Tug-Of-War,Gold,1,1,1866
4,5,Christine Jacoba Aaftink,F,21.0,185.0,82.0,Netherlands,NED,1988,Winter,Calgary,Speed Skating,Speed Skating Women's 500 metres,Thanks,1,2,1967
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
271110,135568,Olga Igorevna Zyuzkova,F,33.0,171.0,69.0,Belarus,BLR,2016,Summer,Rio de Janeiro,Basketball,Basketball Women's Basketball,Thanks,1,1,1983
271111,135569,Andrzej ya,M,29.0,179.0,89.0,Poland-1,POL,1976,Winter,Innsbruck,Luge,Luge Mixed (Men)'s Doubles,Thanks,1,1,1947
271112,135570,Piotr ya,M,27.0,176.0,59.0,Poland,POL,2014,Winter,Sochi,Ski Jumping,"Ski Jumping Men's Large Hill, Individual",Thanks,1,2,1987
271114,135571,Tomasz Ireneusz ya,M,30.0,185.0,96.0,Poland,POL,1998,Winter,Nagano,Bobsleigh,Bobsleigh Men's Four,Thanks,1,1,1968


In [72]:
cleanedData = pd.read_csv('OlympicHistory/CleanedData.csv')
cleanedData.equals(df)

False

In [ ]:
# add a new column for date of birth
#### drop the duplicates with the same age and year.